[Python] [CNN]점자번역 프로그램(3)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
def data_ready(): 
    images_dir = './images' 
    
    datagen = ImageDataGenerator(rotation_range=5, 
                                 shear_range=5, 
                                 validation_split=0.2, 
                                ) #20%를 검증모델로 사용. 
    
    train_generator = datagen.flow_from_directory(images_dir, 
                                                  target_size=(36,36), 
                                                  subset='training')
    
    val_generator = datagen.flow_from_directory(images_dir, 
                                                target_size=(36,36), 
                                                subset='validation') 
    
    return train_generator, val_generator

In [6]:
def load_image(img_path): 
    images_dir = img_path 
    datagen = ImageDataGenerator() 
    real_generator = datagen.flow_from_directory(images_dir, 
                                                 target_size=(36, 36)) 
    
    return real_generator

[Python] [CNN]점자번역 프로그램(4)

In [7]:
from keras import backend as K
from keras import layers as L
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

In [10]:
def Make_model(train,val):
    K.clear_session()

    model_ckpt = ModelCheckpoint('BrailleNet.h5',save_best_only=True)
    reduce_lr = ReduceLROnPlateau(patience=8,verbose=1)
    early_stop = EarlyStopping(patience=20,verbose=2)

    entry = L.Input(shape=(36,36,3))
    x = L.SeparableConv2D(64,(3,3),activation='relu',padding ='same')(entry)
    x = L.MaxPooling2D((2,2))(x)

    x = L.SeparableConv2D(128,(3,3),activation='relu',padding ='same')(x)
    x = L.MaxPooling2D((2,2))(x)

    x = L.SeparableConv2D(256,(2,2),activation='relu',padding ='same')(x)
    x = L.GlobalMaxPooling2D()(x)

    x = L.Dense(256)(x)
    x = L.LeakyReLU()(x)
    x = L.Dense(64,kernel_regularizer=l2(2e-4))(x)
    x = L.LeakyReLU()(x)
    x = L.Dense(27,activation='softmax')(x)

    model = Model(entry,x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    history = model.fit_generator(train,validation_data=val,epochs=666,
                                  callbacks=[model_ckpt,reduce_lr,early_stop],verbose=0)
    return history

In [11]:
def print_acc_loss(history):
    # 평가 결과 도식화
    import matplotlib.pyplot as plt
    fig, loss_ax = plt.subplots(figsize=(10, 5))
    acc_ax = loss_ax.twinx()
    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')
    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

[Python] [CNN]점자번역 프로그램(5)

In [12]:
def load_model():
    from keras.models import load_model
    model = load_model('BrailleNet.h5')
    return model

def acc_chk(model, val):
    acc = model.evaluate_generator(val)[1]
    print('model accuracy: {}'.format(round(acc,4)))

In [14]:
from PIL import Image
import os

class img_devide():
    def __init__(self,img_path):
        self.img_path = img_path
        self.path = ''
        self.call_num = 0
        self.lengh = 0
        self.width = 0
        self.height = 0
        self.img = ''


    def create_dir(self):
        try:
            os.mkdir('./test/a')
            print('create new dir')
        except:
            print('already exist')
            pass
        self.path = './test/a'

    def devide_img(self):
        self.img = Image.open(self.img_path)
        area = (0+self.call_num*self.height,0,self.width/self.lengh*(self.call_num+1),self.height)
        cropped_img = self.img.crop(area)
        cropped_img.save(self.path + '/'+str(self.call_num)+'.jpg')

        self.call_num+=1


    def set_image(self):
        self.img = Image.open(self.img_path)
        self.width = self.img.size[0]
        self.height = self.img.size[1]
        self.lengh = int(self.width / self.height)

    def remove_file(self):
        try:
            os.remove(self.path+'/'+str(self.call_num-1)+'.jpg')
        except:
            pass

    def remove_dir(self) :
        try:
            os.rmdir(self.path)
        except :
            print('fail')

[Python] [CNN]점자번역 프로그램(6)

In [33]:
def alpha(num):

    if num == 26:
        return  ' '
    else:
        num_tr = num+97
        return chr(num_tr)

In [34]:
import operator

class Predic():
    result = []
    def Predict(self,model,real):
        my_list = model.predict(real)
        index, value = max(enumerate(my_list[0]), key=operator.itemgetter(1))
        print(index,alpha(index))
        self.result.append(alpha(index))

        return self.result

    def reset(self):
        self.result = []

def chk_trans():
    for i in range(0, 27):
        print(str(i) + ':' + alpha(i),end='  ')
        if i%3 ==2 :
            print()

In [ ]:
##GPU 오류 해결 (keras_scratch_graph)
#import keraserr
#keraserr.fixerr()

## 이미지 준비 *한번 실행되면 다시 실행할 필요 없음.
# import Rdy_image
Rdy_image.Preset()

## data Generator 테스트, 검증 데이터 생성
# import DATAGenerator
train_generator, val_generator = DATAGenerator.data_ready()

#MAKE MODEL *한번 모델이 생성되면 다시 실행할 필요 없음
# import Make_model
#hist = Make_model.Make_model(train_generator,val_generator)
#Make_model.print_acc_loss(hist)

# BrailleNet에 저장된 모델을 불러옴.
# acc확인
# import Load_model
model = Load_model.load_model()
acc = Load_model.acc_chk(model,val_generator)

# 사진 데이터 불러오기, 예측
# import divide
# import Predict

#path = 'D:/ML/Project_f/real/a/l.jpg'

def action(path):

    Predict.chk_trans()
    b = Predict.Predic()
    a = divide.img_devide(path)

    a.create_dir()
    a.set_image()
    b.reset()

    for i in range(0,a.lengh):
        a.devide_img()
        real = DATAGenerator.load_image('./test')
        b.Predict(model,real)
        a.remove_file()


    print(b.result)
    result = ''.join(b.result)
    return result